# ENCODE FILES DO NOT ALWAYS MATCH THE RBP NAME

In [1]:
import glob
import functools
import urllib2
import datetime
import pandas as pd
import json
import gspread
import pandas as pd
import os
from IPython.core.display import HTML
from oauth2client.client import SignedJwtAssertionCredentials
import numpy as np
import shutil
from collections import Counter
import re
import time
pd.set_option("display.max_colwidth", 1000)
pd.set_option('display.max_columns', 500)

In [2]:
import oauth2client
print('{}, {}'.format(gspread.__version__, oauth2client.__version__))
# needs to be 0.6.2 and 1.5.2 (most recent versions won't work)

0.6.2, 1.5.2


# This notebook provides basic functionality for accessing Gabe's old eCLIP paired-end spreadsheet.

In [3]:
date = datetime.date.today().isoformat()
input_dir = '/projects/ps-yeolab3/bay001/encode/'
output_dir = '/projects/ps-yeolab3/bay001/encode/'
json_file = "/home/gpratt/ipython_notebook/public clip-588adbc137f3.json"
gsheet_url = "https://docs.google.com/spreadsheets/d/1ZU2mQh54jentqvhR_oMnviLGWR8Nw_x338gULzKjNDI/edit#gid=0"

# Necessary functions

In [4]:

def format_file(index, adapter):
    """
    - index: row from dataframe that represents the manifest
    - adapter: R1 or R2 for the reads
    - returns: location on TSCC
    """
    index_1 = illumina_adapters.ix[index.index_1].values[0]
    index_2 = illumina_adapters.ix[index.index_2].values[0]
    if index.Lane == "": #Incase we are doing a rapid run and there isn't lane info
        sample_name = "{}_{}-{}_{}.fastq.gz".format(index.Hiseq_file_name, index_2, index_1, adapter)
        dir_name = "Sample_{}".format(index.Hiseq_file_name)

    #For Sebastians samples Sample_G3BP-S-input/G3BP-S-input_S1_L001_R1.fastq.gz
    elif index.Lane.startswith("S") and "L" in index.Lane:
        name = index.Hiseq_file_name
        sample_name = "{}_{}_{}.fastq.gz".format(name, index.Lane, adapter)
        dir_name = index.Hiseq_file_name
    
    elif index.Lane.startswith("S"):
        name = index.Hiseq_file_name.split("-")[0].replace("_", "-")
        sample_name = "{}_{}_{}.fastq.gz".format(name, index.Lane, adapter)
        dir_name = index.Hiseq_file_name
 
    #This needs to go above my other 4000 check because of name colisions
    elif index.is_4000:
        sample_name = "{}_{}_{}_001.fastq.gz".format(index.Hiseq_file_name, index.Lane, adapter)
        return os.path.join(index.file_location, sample_name)
    
    #this is a bit hacky, keep an eye on it
    elif "4000" in index.Hiseq_file_name:
        sample_name = index.Hiseq_file_name + ".fastq.gz"
        sample_name = sample_name.replace("R1", adapter)
        return os.path.join(index.file_location, sample_name)
      
    else:
        sample_name = "{}_{}-{}_{}_{}.fastq.gz".format(index.Hiseq_file_name, index_2, index_1, index.Lane, adapter)
        dir_name = "Sample_{}".format(index.Hiseq_file_name)
    return os.path.join(index.file_location, dir_name, sample_name)

In [5]:
try:
    illumina_adapters = """501 TATAGCCT 
    502 ATAGAGGC 
    503 CCTATCCT 
    504 GGCTCTGA 
    505 AGGCGAAG 
    506 TAATCTTA 
    507 CAGGACGT 
    508 GTACTGAC 
    502s ATA 
    503s CCT
    504s GGC
    505,506 eric_is_stupid
    505s AGG
    506s TAA
    507s CAG
    508s GTA
    701 ATTACTCG
    702 TCCGGAGA
    703 CGCTCATT
    704 GAGATTCC
    705 ATTCAGAA
    706 GAATTCGT""".split("\n")

    illumina_adapters = pd.DataFrame([item.strip().split() for item in illumina_adapters], columns=["label", "barcode"])
    illumina_adapters = illumina_adapters.set_index("label")

    json_key = json.load(open(json_file))
    scope = ['https://spreadsheets.google.com/feeds']

    credentials = SignedJwtAssertionCredentials(json_key['client_email'], json_key['private_key'], scope)
    gc = gspread.authorize(credentials)

    sht1 = gc.open_by_url(gsheet_url)
    ws = sht1.worksheet("Sheet1")
    list_of_lists = ws.get_all_values()
    manifest = pd.DataFrame(list_of_lists[1:], columns=list_of_lists[0])
    manifest.is_encode = manifest.is_encode == "TRUE"
    manifest.is_4000 = manifest.is_4000 == "TRUE"

    manifest.method_Paper_flag = manifest.Method_Paper_flag == "TRUE" 
    manifest['exp_id'] = manifest.ENCODE_ID.apply(lambda x: x.split("_")[0])

    manifest['r1'] = manifest.apply(functools.partial(format_file, adapter="R1"), axis=1)
    manifest['r2'] = manifest.apply(functools.partial(format_file, adapter="R2"), axis=1)
except KeyError as e:
    print e

/projects/ps-yeolab4/bay001/anaconda2/envs/encode3-manifest/lib/python2.7/site-packages/ipykernel_launcher.py:41: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
/projects/ps-yeolab4/bay001/anaconda2/envs/encode3-manifest/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/projects/ps-yeolab4/bay001/anaconda2/envs/encode3-manifest/lib/python2.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


In [6]:
print(manifest.shape) # number of samples
manifest.head()

(2272, 28)


,Hiseq_file_name,ENCODE_ID,RBP,inline_1,inline_2,index_1,index_2,Lane,file_location,unmerged_location,original_file_name,is_encode,cell_type,hiseq_run_date,randomer_length,,Method_Paper_flag,species,is_4000,,,,,,,exp_id,r1,r2
0,222_CLIP,222_01,HNRNPM,C01,D08fixed,503,701,L001,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/,,,True,HepG2,,5,,TRUE,hg19,False,,,,,,,222,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/Sample_222_CLIP/222_CLIP_ATTACTCG-CCTATCCT_L001_R1.fastq.gz,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/Sample_222_CLIP/222_CLIP_ATTACTCG-CCTATCCT_L001_R2.fastq.gz
1,222_CLIP,222_02,HNRNPM,A01,B06,503,701,L001,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/,,,True,HepG2,,5,,TRUE,hg19,False,,,,,,,222,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/Sample_222_CLIP/222_CLIP_ATTACTCG-CCTATCCT_L001_R1.fastq.gz,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/Sample_222_CLIP/222_CLIP_ATTACTCG-CCTATCCT_L001_R2.fastq.gz
2,222_INPUT,222_INPUT,HNRNPM,none,none,502,701,L001,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/,,,True,HepG2,,5,,TRUE,hg19,False,,,,,,,222,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/Sample_222_INPUT/222_INPUT_ATTACTCG-ATAGAGGC_L001_R1.fastq.gz,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/Sample_222_INPUT/222_INPUT_ATTACTCG-ATAGAGGC_L001_R2.fastq.gz
3,223_CLIP,223_01,FKBP4,A01,B06,503,702,L001,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/,,,True,HepG2,,5,,TRUE,hg19,False,,,,,,,223,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/Sample_223_CLIP/223_CLIP_TCCGGAGA-CCTATCCT_L001_R1.fastq.gz,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/Sample_223_CLIP/223_CLIP_TCCGGAGA-CCTATCCT_L001_R2.fastq.gz
4,223_CLIP,223_02,FKBP4,C01,D08fixed,503,702,L001,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/,,,True,HepG2,,5,,TRUE,hg19,False,,,,,,,223,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/Sample_223_CLIP/223_CLIP_TCCGGAGA-CCTATCCT_L001_R1.fastq.gz,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150406_D00611_0100_AC6DB4ANXX/Data/Unaligned/Project_AC6DB4ANXX/Sample_223_CLIP/223_CLIP_TCCGGAGA-CCTATCCT_L001_R2.fastq.gz


In [7]:
output_file = os.path.join(output_dir, '{}_eCLIP_manifest.txt'.format(date))
manifest.to_csv(
    output_file, sep='\t'
)

In [8]:
manifest[(manifest['ENCODE_ID'].str.contains('204')==True)]

,Hiseq_file_name,ENCODE_ID,RBP,inline_1,inline_2,index_1,index_2,Lane,file_location,unmerged_location,original_file_name,is_encode,cell_type,hiseq_run_date,randomer_length,,Method_Paper_flag,species,is_4000,,,,,,,exp_id,r1,r2
108,293XT_Input_0204,293XT_Input_0204,Input,none,none,506,705,,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150414_D00611_0103_AC6FLTANXX/Unaligned_Yeo/Project_C6FLTANXX,,,False,293XT,,5,,TRUE,hg19,False,,,,,,,293XT,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150414_D00611_0103_AC6FLTANXX/Unaligned_Yeo/Project_C6FLTANXX/Sample_293XT_Input_0204/293XT_Input_0204_ATTCAGAA-TAATCTTA_R1.fastq.gz,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150414_D00611_0103_AC6FLTANXX/Unaligned_Yeo/Project_C6FLTANXX/Sample_293XT_Input_0204/293XT_Input_0204_ATTCAGAA-TAATCTTA_R2.fastq.gz
111,293XT_CLIP_RBFOX2_0204,293XT_CLIP_RBFOX2_0204,RBFOX2,A03,G07,506,704,,/home/gpratt/projects/encode/data/encode_merged/,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150414_D00611_0103_AC6FLTANXX/Unaligned_Yeo/Project_C6FLTANXX,,False,293XT,,5,,TRUE,hg19,False,,,,,,,293XT,/home/gpratt/projects/encode/data/encode_merged/Sample_293XT_CLIP_RBFOX2_0204/293XT_CLIP_RBFOX2_0204_GAGATTCC-TAATCTTA_R1.fastq.gz,/home/gpratt/projects/encode/data/encode_merged/Sample_293XT_CLIP_RBFOX2_0204/293XT_CLIP_RBFOX2_0204_GAGATTCC-TAATCTTA_R2.fastq.gz
158,RBFOX2_204_CLIP-22583561,204_01,RBFOX2,A01,B06,501,702,S1,/projects/ps-yeolab/seqdata/20150224_encode_CLIP/SN1001_0449-20536517,,,True,HepG2,,5,,TRUE,hg19,False,,,,,,,204,/projects/ps-yeolab/seqdata/20150224_encode_CLIP/SN1001_0449-20536517/RBFOX2_204_CLIP-22583561/RBFOX2-204-CLIP_S1_R1.fastq.gz,/projects/ps-yeolab/seqdata/20150224_encode_CLIP/SN1001_0449-20536517/RBFOX2_204_CLIP-22583561/RBFOX2-204-CLIP_S1_R2.fastq.gz
159,RBFOX2_204_CLIP-22583561,204_02,RBFOX2,C01,D08fixed,501,702,S1,/projects/ps-yeolab/seqdata/20150224_encode_CLIP/SN1001_0449-20536517,,,True,HepG2,,5,,TRUE,hg19,False,,,,,,,204,/projects/ps-yeolab/seqdata/20150224_encode_CLIP/SN1001_0449-20536517/RBFOX2_204_CLIP-22583561/RBFOX2-204-CLIP_S1_R1.fastq.gz,/projects/ps-yeolab/seqdata/20150224_encode_CLIP/SN1001_0449-20536517/RBFOX2_204_CLIP-22583561/RBFOX2-204-CLIP_S1_R2.fastq.gz
160,RBFOX2_204_INPUT-22583562,204_INPUT,RBFOX2,none,none,501,704,S2,/projects/ps-yeolab/seqdata/20150224_encode_CLIP/SN1001_0449-20536517,,,True,HepG2,,5,,TRUE,hg19,False,,,,,,,204,/projects/ps-yeolab/seqdata/20150224_encode_CLIP/SN1001_0449-20536517/RBFOX2_204_INPUT-22583562/RBFOX2-204-INPUT_S2_R1.fastq.gz,/projects/ps-yeolab/seqdata/20150224_encode_CLIP/SN1001_0449-20536517/RBFOX2_204_INPUT-22583562/RBFOX2-204-INPUT_S2_R2.fastq.gz
547,293XT_RBFOX2_0204_CLIP_S1_L006_R1_001_4000,293XT_RBFOX2_0204_CLIP_4000,RBFOX2,A03,G07,506,704,L006,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150929_K00180_0055_AH3NHMBBXX/Unaligned_Yeo,,,False,,,5,,FALSE,hg19,False,,,,,,,293XT,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150929_K00180_0055_AH3NHMBBXX/Unaligned_Yeo/293XT_RBFOX2_0204_CLIP_S1_L006_R1_001_4000.fastq.gz,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150929_K00180_0055_AH3NHMBBXX/Unaligned_Yeo/293XT_RBFOX2_0204_CLIP_S1_L006_R2_001_4000.fastq.gz
548,293XT_RBFOX2_0204_INPUT_S3_L006_R1_001_4000,293XT_RBFOX2_0204_INPUT_4000,RBFOX2,none,none,506,705,L006,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150929_K00180_0055_AH3NHMBBXX/Unaligned_Yeo,,,False,,,5,,FALSE,hg19,False,,,,,,,293XT,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150929_K00180_0055_AH3NHMBBXX/Unaligned_Yeo/293XT_RBFOX2_0204_INPUT_S3_L006_R1_001_4000.fastq.gz,/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/150929_K00180_0055_AH3NHMBBXX/Unaligned_Yeo/293XT_RBFOX2_0204_INPUT_S3_L006_R2_001_4000.fastq.gz
